In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

在[Vertex AI](https://cloud.google.com/vertex-ai)上部署PyTorch文本分类模型

**在运行任何规模测试或有任何问题之前，请务必与Vertex AI联系。**

# 概述

本示例重点介绍如何使用Vertex AI Prediction预构建的PyTorch图像，在Vertex AI上部署PyTorch文本分类模型。本示例假设您已经在GCS uri上拥有一个训练过的模型。如果您还没有训练过PyTorch文本分类模型，可以按照[此笔记本](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/community-content/pytorch_text_classification_using_vertex_sdk_and_gcloud/pytorch-text-classification-vertex-ai-train-tune-deploy.ipynb) 进行训练。

### 目标

如何在[Vertex AI](https://cloud.google.com/vertex-ai)上部署PyTorch模型，并突出PyTorch模型在Vertex AI上部署的支持。在本笔记本中，您不会关注在Vertex AI上训练PyTorch模型的支持。查看 [此笔记本](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/community-content/pytorch_text_classification_using_vertex_sdk_and_gcloud/pytorch-text-classification-vertex-ai-train-tune-deploy.ipynb) 了解更多有关在Vertex AI上训练支持的信息。

### 内容目录

本笔记本包括以下部分:

- [Creating Notebooks instance](#Creating-Notebooks-instance-on-Google-Cloud)
- [Deploying](#Deploying)
    - [Deploying model on Vertex AI Predictions with custom container](#Deploying-the-pre-built-PyTorch-container-to-Vertex-AI-Predictions)

### 成本

本教程使用Google Cloud Platform（GCP）的计费组件：

* [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench)
* [Vertex AI Predictions](https://cloud.google.com/vertex-ai/docs/predictions/getting-predictions)
* [Cloud Storage](https://cloud.google.com/storage)

了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage定价](https://cloud.google.com/storage/pricing)，并使用[定价计算器](https://cloud.google.com/products/calculator/)根据您的预期使用情况生成成本估算。

### 设置您的本地开发环境

**如果您在使用Colab或Google Cloud笔记本电脑**，您的环境已经满足
运行此笔记本的所有要求。您可以跳过此步骤。

否则，请确保您的环境符合本笔记本的要求。
您需要以下内容：

• Google Cloud SDK
• Git
• Python 3
• virtualenv
• 在使用Python 3的虚拟环境中运行Jupyter笔记本

Google Cloud的[设置Python开发环境指南](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了详细的说明来满足这些要求。以下步骤提供了一个简化的指南：

1. [安装并初始化Cloud SDK。](https://cloud.google.com/sdk/docs/)
2. [安装Python 3。](https://cloud.google.com/python/setup#installing_python)
3. 安装virtualenv并创建一个使用Python 3的虚拟环境。激活虚拟环境。
4. 要安装Jupyter，请在终端shell中的命令行中运行`pip3 install jupyter`。
5. 要启动Jupyter，请在终端shell中的命令行中运行`jupyter notebook`。
6. 在Jupyter Notebook仪表板中打开这个笔记本。

### 安装额外的包

此笔记本需要的Python依赖包包括[Torch](https://pypi.org/project/torch/)和[Torch Model Archiver](https://pypi.org/project/torch-model-archiver/)，它们将在笔记本实例中安装。

In [ ]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

In [ ]:
!pip install {USER_FLAG} --upgrade torch==1.11

你将使用[Python的Vertex AI SDK](https://cloud.google.com/vertex-ai/docs/start/client-libraries#python)与Vertex AI服务进行交互。高级别的`aiplatform`库旨在通过使用包装类和偏见的默认值来简化常见的数据科学工作流程。

#### 安装Python的Vertex AI SDK

In [ ]:
!pip install {USER_FLAG} --upgrade google-cloud-aiplatform[prediction]

#### 安装 Torch Model Archiver
如果您想要使用 Vertex AI 预构建的 PyTorch 图像在 Vertex AI 上部署训练好的模型，这将是必需的。

In [ ]:
!pip install {USER_FLAG} --upgrade torch-model-archiver

### 重新启动内核

在安装了额外的包之后，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

在你开始之前

### 设置您的Google Cloud项目

**无论您使用的是哪种笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建账户时，您将获得300美元的免费信用额度用于计算/存储成本。
1. [确保为您的项目启用了计费](https://cloud.google.com/billing/docs/how-to/modify-project)。
1. 在您的项目中启用以下API，这些API对于运行教程是必需的
    - [Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)
    - [Cloud Storage API](https://console.cloud.google.com/flows/enableapi?apiid=storage.googleapis.com)
1. 如果您正在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。
1. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter会将以`!`为前缀的行视为shell命令，并将以`$`为前缀的Python变量插入到这些命令中。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`或`google.auth`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # <---CHANGE THIS TO YOUR PROJECT

import os

# Get your Google Cloud project ID using google.auth
if not os.getenv("IS_TESTING"):
    import google.auth

    _, PROJECT_ID = google.auth.default()
    print("Project ID: ", PROJECT_ID)

# validate PROJECT_ID
if PROJECT_ID == "" or not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    print(
        f"Please set your project id before proceeding to next step. Currently it's set as {PROJECT_ID}"
    )

时间戳

如果您在一个现场教程会话中，您可能在使用一个共享的测试帐户或项目。为了避免用户在创建的资源之间发生名称冲突，您为每个实例会话创建一个时间戳，并将其附加到您在这个教程中创建的资源的名称上。

In [ ]:
from datetime import datetime


def get_timestamp():
    return datetime.now().strftime("%Y%m%d%H%M%S")


TIMESTAMP = get_timestamp()
print(f"TIMESTAMP = {TIMESTAMP}")

### 认证您的Google Cloud账户

---

**如果您正在使用Google Cloud笔记本**，您的环境已经通过认证。请跳过此步骤。

**如果你正在使用Colab**，运行下面的单元格，并按提示进行操作，通过oAuth验证你的帐号。

**否则**，请按照以下步骤操作：

1. 在 Cloud 控制台中，转到[**创建服务帐号密钥**页面](https://console.cloud.google.com/apis/credentials/serviceaccountkey)。
2. 点击**创建服务帐号**。
3. 在**服务帐号名称**字段中输入名称，然后点击**创建**。
4. 在**授予此服务帐号对项目的访问权限**部分，点击**角色**下拉列表。在过滤框中键入“Vertex AI”，选择 **Vertex AI管理员**。在过滤框中键入“存储对象管理员”，选择**存储对象管理员**。
5. 点击*创建*。一个包含你的密钥的JSON文件将下载到你的本地环境。
6. 在下面的单元格中输入你的服务帐号密钥的路径作为`GOOGLE_APPLICATION_CREDENTIALS`变量，然后运行该单元格。

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

创建一个云存储桶

**无论您使用什么笔记本环境，以下步骤都是必需的。**

在下面设置您的云存储桶的名称。它必须在所有云存储桶中是唯一的。

您还可以更改`REGION`变量，该变量用于本笔记本其余部分的操作。请确保[选择一个 Vertex AI 服务可用的区域](https://cloud.google.com/vertex-ai/docs/general/locations#available_regions)。您不能使用多区域存储桶来进行 Vertex AI 的预测。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # <---CHANGE THIS TO YOUR BUCKET
REGION = "us-central1"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or not BUCKET_NAME or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = f"gs://{PROJECT_ID}aip-{get_timestamp()}"

In [ ]:
print(f"PROJECT_ID = {PROJECT_ID}")
print(f"BUCKET_NAME = {BUCKET_NAME}")
print(f"REGION = {REGION}")

只有在您的存储桶尚不存在时：运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查Cloud Storage存储桶的内容验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

部署

在[Vertex AI](https://cloud.google.com/vertex-ai/docs/predictions/getting-predictions)上部署PyTorch模型以提供在线预测时，无需创建自定义容器。您可以部署Vertex AI Prediction预构建的PyTorch容器，以便从Hugging Face Transformers fine-tuned transformer模型中提供情感分析任务的预测。然后，您可以使用Vertex AI对输入文本进行情感分类。

基本上，要使用Vertex AI Prediction预构建的PyTorch图像在Vertex AI上部署PyTorch模型，需按以下步骤操作：

1. 打包训练的模型文件，包括[默认](https://pytorch.org/serve/#default-handlers)或[自定义](https://pytorch.org/serve/custom_service.html)处理程序，通过使用[Torch模型归档器](https://github.com/pytorch/serve/tree/master/model-archiver)创建一个归档文件。
2. 在本地运行带有模型文件的预构建PyTorch图像（可选）。
3. 上传带有预构建PyTorch图像的模型，以作为Vertex AI模型资源提供预测服务。
4. 创建一个Vertex AI端点，并[部署模型](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)资源。

下载模型工件

从云存储下载作为训练（或超参数调整）作业的一部分保存的模型工件到本地目录。如果您尚未拥有经过训练的PyTorch文本分类模型，请参考[此笔记本](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/community-content/pytorch_text_classification_using_vertex_sdk_and_gcloud/pytorch-text-classification-vertex-ai-train-tune-deploy.ipynb)。

In [ ]:
GCS_TRAINED_MODEL_URI = "gs://[your-gcs-path]"  # <---CHANGE THIS TO YOUR GCS PATH THAT CONTAINS MODEL ARTIFACTS

验证云存储桶中的模型文件。

In [ ]:
!gsutil ls -r $GCS_TRAINED_MODEL_URI

从云存储复制文件到本地目录。

In [ ]:
!mkdir trained_model
!gsutil -m cp -r $GCS_TRAINED_MODEL_URI/ ./trained_model

In [ ]:
!ls -ltrR ./trained_model

In [ ]:
LOCAL_TRAINED_MODEL_DIRECTORY = "[your-local-directory]"  # <---CHANGE THIS TO YOUR LOCAL DIRECTORY THAT CONTAINS MODEL ARTIFACTS

创建自定义模型处理程序来处理预测请求

在使用经过精细调整的转换器模型预测输入文本的情感时，需要对输入文本进行预处理，并通过将名称（积极/消极）添加到目标标签（1/0）以及概率（或置信度）进行后处理。您可以创建一个自定义处理程序脚本，该脚本打包在模型工件中，并且预构建的PyTorch图像在运行时执行该代码。

自定义处理程序脚本执行以下操作：

- 在将输入文本发送到模型进行推理之前，对其进行预处理
- 自定义调用模型进行推理的方式
- 在发送回响应之前，从模型中对输出进行后处理

请参考[TorchServe文档](https://pytorch.org/serve/custom_service.html)来定义一个自定义处理程序。

In [ ]:
PREDICTOR_DIRECTORY = "./predictor"

!mkdir $PREDICTOR_DIRECTORY

In [ ]:
%%writefile $PREDICTOR_DIRECTORY/custom_handler.py

import os
import json
import logging

import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from ts.torch_handler.base_handler import BaseHandler

logger = logging.getLogger(__name__)


class TransformersClassifierHandler(BaseHandler):
    """
    The handler takes an input string and returns the classification text 
    based on the serialized transformers checkpoint.
    """
    def __init__(self):
        super(TransformersClassifierHandler, self).__init__()
        self.initialized = False

    def initialize(self, ctx):
        """ Loads the model.pt file and initialized the model object.
        Instantiates Tokenizer for preprocessor to use
        Loads labels to name mapping file for post-processing inference response
        """
        self.manifest = ctx.manifest

        properties = ctx.system_properties
        model_dir = properties.get("model_dir")
        self.device = torch.device("cuda:" + str(properties.get("gpu_id")) if torch.cuda.is_available() else "cpu")

        # Read model serialize/pt file
        serialized_file = self.manifest["model"]["serializedFile"]
        model_pt_path = os.path.join(model_dir, serialized_file)
        if not os.path.isfile(model_pt_path):
            raise RuntimeError("Missing the model.pt or pytorch_model.bin file")
        
        # Load model
        self.model = AutoModelForSequenceClassification.from_pretrained(model_dir)
        self.model.to(self.device)
        self.model.eval()
        logger.debug('Transformer model from path {0} loaded successfully'.format(model_dir))
        
        # Ensure to use the same tokenizer used during training
        self.tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

        # Read the mapping file, index to object name
        mapping_file_path = os.path.join(model_dir, "index_to_name.json")

        if os.path.isfile(mapping_file_path):
            with open(mapping_file_path) as f:
                self.mapping = json.load(f)
        else:
            logger.warning('Missing the index_to_name.json file. Inference output will default.')
            self.mapping = {"0": "Negative",  "1": "Positive"}

        self.initialized = True

    def preprocess(self, data):
        """ Preprocessing input request by tokenizing
            Extend with your own preprocessing steps as needed
        """
        text = data[0].get("data")
        if text is None:
            text = data[0].get("body")
        sentences = text.decode('utf-8')
        logger.info("Received text: '%s'", sentences)

        # Tokenize the texts
        tokenizer_args = ((sentences,))
        inputs = self.tokenizer(*tokenizer_args,
                                padding='max_length',
                                max_length=128,
                                truncation=True,
                                return_tensors = "pt")
        return inputs

    def inference(self, inputs):
        """ Predict the class of a text using a trained transformer model.
        """
        prediction = self.model(inputs['input_ids'].to(self.device))[0].argmax().item()

        if self.mapping:
            prediction = self.mapping[str(prediction)]

        logger.info("Model predicted: '%s'", prediction)
        return [prediction]

    def postprocess(self, inference_output):
        return inference_output


生成目标标签以命名文件

在自定义处理程序中，您要引用一个目标标签与其有意义名称之间的映射文件，用于格式化预测响应。在这里，您将目标标签“0”映射为“Negative”，将“1”映射为“Positive”。

In [ ]:
%%writefile $PREDICTOR_DIRECTORY/index_to_name.json

{
    "0": "Negative", 
    "1": "Positive"
}

打包已训练的模型文件

预构建的PyTorch图像需要使用[Torch模型打包工具](https://github.com/pytorch/serve/tree/master/model-archiver)创建模型存档文件。

In [ ]:
ARCHIVED_MODEL_PATH = "./archived_model"

!mkdir $ARCHIVED_MODEL_PATH

In [ ]:
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires to add a path to find the installed torch-model-archiver
if IS_GOOGLE_CLOUD_NOTEBOOK:
    os.environ["PATH"] = f'{os.environ.get("PATH")}:~/.local/bin'

将经过训练的模型工件打包，包括[默认](https://pytorch.org/serve/#default-handlers)或[自定义](https://pytorch.org/serve/custom_service.html)处理程序，通过使用[Torch模型存档工具](https://github.com/pytorch/serve/tree/master/model-archiver)创建一个存档文件。预构建的PyTorch映像需要模型存档文件命名为`model.mar`，因此您需要将模型名称设置为`model`。

In [ ]:
!torch-model-archiver -f \
  --model-name=model \
  --version=1.0 \
  --serialized-file=$LOCAL_TRAINED_MODEL_DIRECTORY/pytorch_model.bin \
  --handler=$PREDICTOR_DIRECTORY/custom_handler.py \
  --extra-files "$LOCAL_TRAINED_MODEL_DIRECTORY/config.json,$LOCAL_TRAINED_MODEL_DIRECTORY/tokenizer.json,$LOCAL_TRAINED_MODEL_DIRECTORY/training_args.bin,$LOCAL_TRAINED_MODEL_DIRECTORY/tokenizer_config.json,$LOCAL_TRAINED_MODEL_DIRECTORY/special_tokens_map.json,$LOCAL_TRAINED_MODEL_DIRECTORY/vocab.txt,$PREDICTOR_DIRECTORY/index_to_name.json" \
  --export-path=$ARCHIVED_MODEL_PATH

在本地运行预先构建好的 PyTorch 图像（可选）

在将模型上传到 Vertex AI 之前，您可以在本地运行带有模型文件的预先构建的 PyTorch 图像。

Vertex AI 预测在不同的 [Artifact Registry 多区域](https://cloud.google.com/artifact-registry/docs/repositories/repo-locations) 中具有预先构建的图像。要在本地运行这些图像，您可以使用与目标地区相匹配的图像。在此示例中，您使用的是在 CPU 上的 PyTorch 1.11，因此您可以选择下列任一图像：
- us-docker.pkg.dev/vertex-ai/prediction/pytorch-cpu.1-11:latest
- europe-docker.pkg.dev/vertex-ai/prediction/pytorch-cpu.1-11:latest
- asia-docker.pkg.dev/vertex-ai/prediction/pytorch-cpu.1-11:latest

Vertex AI 提供 [Vertex SDK](https://github.com/googleapis/python-aiplatform/tree/main/google/cloud/aiplatform/prediction) 来帮助在本地测试图像。您将使用 Vertex SDK 来测试带有归档模型文件的预构建的 PyTorch 图像。

在笔记本中设置日志配置。

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)

选择与模型将部署到的地区匹配的图像URI。有关Artifact Registry多地区的详细信息，请查看[文档](https://cloud.google.com/artifact-registry/docs/repositories/repo-locations)。

In [ ]:
serving_container_image_uri = "[your-multi-region]-docker.pkg.dev/vertex-ai/prediction/pytorch-cpu.1-11:latest"  # <---CHANGE THIS TO YOUR MULTI REGION, COULD BE `us`, `europe`, or `asia`

由于您正在本地使用预构建的PyTorch图像，您需要填充必要的路由和端口。在使用预构建的PyTorch图像上传模型到Vertex AI时，您不需要填充路由或端口。

In [ ]:
health_route = "/ping"
predict_route = "/predictions/model"
serving_container_ports = [8080]

创建本地模型。

In [ ]:
from google.cloud.aiplatform.prediction import LocalModel

local_model = LocalModel(
    serving_container_image_uri=serving_container_image_uri,
    serving_container_predict_route=predict_route,
    serving_container_health_route=health_route,
    serving_container_ports=serving_container_ports,
)

存储测试实例。要了解有关在JSON中格式化输入实例的更多信息，请阅读文档。

In [ ]:
INPUT_FILE = "./instances.json"

In [ ]:
%%bash

cat > ./instances.json <<END
{ 
   "instances": [
     { 
       "data": {
         "b64": "$(echo 'Take away the CGI and the A-list cast and you end up with film with less punch.' | base64 --wrap=0)"
       }
     }
   ]
}
END

在本地运行并发送请求给容器。在这个测试中，您需要运行健康检查和预测请求。

In [ ]:
with local_model.deploy_to_local_endpoint(
    artifact_uri=f"{ARCHIVED_MODEL_PATH}",
) as local_endpoint:
    health_check_response = local_endpoint.run_health_check()

    predict_response = local_endpoint.predict(
        request_file=INPUT_FILE,
        headers={"Content-Type": "application/json"},
    )

打印出健康检查响应及其内容。

In [ ]:
print(health_check_response, health_check_response.content)

打印出预测的响应及其内容。

In [ ]:
print(predict_response, predict_response.content)

也打印出所有的容器日志。

In [ ]:
local_endpoint.print_container_logs(show_all=True)

部署预构建的PyTorch容器至Vertex AI预测

您在Vertex AI上创建一个模型资源，并将模型部署到Vertex AI端点。在使用模型之前，您必须将模型部署到端点。部署的模型会运行预构建的PyTorch图像来提供预测。

In [ ]:
ARCHIVED_MODEL_GCS_URI = f"{BUCKET_NAME}/archived-pytorch-model"

复制存档模型到GCS。

In [ ]:
!gsutil cp -r $ARCHIVED_MODEL_PATH $ARCHIVED_MODEL_GCS_URI

验证存档的模型文件是否存在于云存储桶中。要部署PyTorch模型使用Vertex AI Prediction预构建的PyTorch图像，您必须在artifact URI下具有一个`model.mar`文件。

In [ ]:
! gsutil ls -al $ARCHIVED_MODEL_GCS_URI

##### **初始化 Python 的 Vertex AI SDK**

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

使用预先构建的PyTorch镜像创建一个模型资源.

In [ ]:
VERSION = 1
model_display_name = f"pytorch-v{VERSION}-{TIMESTAMP}"
model_description = "PyTorch based text classifier with the pre-built PyTorch image"

选项1：通过LocalModel创建一个Model资源（如果你已经在本地运行了容器）

In [ ]:
model = aiplatform.Model.upload(
    local_model=local_model,
    display_name=model_display_name,
    description=model_description,
    artifact_uri=ARCHIVED_MODEL_GCS_URI,
)

model.wait()

print(model.display_name)
print(model.resource_name)

选项2. 通过Model在Model中创建一个Model资源（如果您尚未在本地运行容器）。

In [ ]:
serving_container_image_uri = "[your-multi-region]-docker.pkg.dev/vertex-ai/prediction/pytorch-cpu.1-11:latest"  # <---CHANGE THIS TO YOUR MULTI REGION, COULD BE `us`, `europe`, or `asia`

In [ ]:
model = aiplatform.Model.upload(
    display_name=model_display_name,
    description=model_description,
    serving_container_image_uri=serving_container_image_uri,
    artifact_uri=ARCHIVED_MODEL_GCS_URI,
)

model.wait()

print(model.display_name)
print(model.resource_name)

有关上传或导入模型的更多上下文，请参考[文档](https://cloud.google.com/vertex-ai/docs/general/import-model)。

使用预构建的PyTorch镜像为模型创建端点

In [ ]:
endpoint_display_name = f"pytorch-endpoint-{TIMESTAMP}"
endpoint = aiplatform.Endpoint.create(display_name=endpoint_display_name)

部署模型到端点

部署模型将物理资源与模型关联起来，以便可以通过低延迟提供在线预测。

**注意：** 这一步需要几分钟时间来部署资源。

In [ ]:
traffic_percentage = 100
machine_type = "n1-standard-4"
deployed_model_display_name = model_display_name
sync = True

endpoint = model.deploy(
    endpoint=endpoint,
    deployed_model_display_name=deployed_model_display_name,
    machine_type=machine_type,
    traffic_percentage=traffic_percentage,
    sync=sync,
)

使用Vertex AI SDK调用部署模型的端点进行预测

列出端点上部署的模型

In [ ]:
endpoint.list_models()

将在线预测输入格式化

这个笔记本使用[Torchserve的KServe基于推理API](https://pytorch.org/serve/inference_api.html#kserve-inference-api)，同时也符合[Vertex AI Predictions兼容格式](https://cloud.google.com/vertex-ai/docs/predictions/custom-container-requirements#prediction)。对于在线预测请求，需要将预测输入实例格式化为JSON，并进行base64编码，如下所示：

```
[
    {
        "data": {
            "b64": "<base64编码的字符串>"
        }
    }
]
```

定义样本文本以测试预测

In [ ]:
test_instances = [
    b"Jaw dropping visual affects and action! One of the best I have seen to date.",
    b"Take away the CGI and the A-list cast and you end up with film with less punch.",
]

发送在线预测请求

格式化输入文本字符串，并使用格式化的输入请求调用预测终端点，获取响应

In [ ]:
import base64
import json

print("=" * 100)
for instance in test_instances:
    print(f"Input text: \n\t{instance.decode('utf-8')}\n")
    b64_encoded = base64.b64encode(instance)
    test_instance = [{"data": {"b64": f"{str(b64_encoded.decode('utf-8'))}"}}]
    print(f"Formatted input: \n{json.dumps(test_instance, indent=4)}\n")
    prediction = endpoint.predict(instances=test_instance)
    print(f"Prediction response: \n\t{prediction}")
    print("=" * 100)

##### **【可选】**使用gcloud CLI发出预测请求
您还可以使用[`gcloud beta ai endpoints predict`](https://cloud.google.com/sdk/gcloud/reference/beta/ai/endpoints/predict)调用Vertex AI Endpoint来进行预测。

以下是如何使用`gcloud` CLI向Vertex AI Endpoints发出预测请求的示例：

In [ ]:
%%bash -s $REGION $endpoint_display_name

REGION=$1
endpoint_display_name=$2

# get endpoint id
echo "REGION = ${REGION}"
echo "ENDPOINT DISPLAY NAME = ${endpoint_display_name}"
endpoint_id=$(gcloud beta ai endpoints list --region ${REGION} --filter "display_name=${endpoint_display_name}" --format "value(ENDPOINT_ID)")
echo "ENDPOINT_ID = ${endpoint_id}"

# call prediction endpoint
input_text="Take away the CGI and the A-list cast and you end up with film with less punch."
echo "INPUT TEXT = ${input_text}"

prediction=$(
echo """
{ 
   "instances": [
     { 
       "data": {
         "b64": "$(echo ${input_text} | base64 --wrap=0)"
       }
     }
   ]
}
""" | gcloud beta ai endpoints predict ${endpoint_id} --region=$REGION --json-request -)

echo "PREDICTION RESPONSE = ${prediction}"

## 清理工作

### 清理训练和部署资源

要清理此笔记中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 模型
- 端点
- 云存储桶

将要删除的资源类型设置为标记。

In [ ]:
delete_endpoint = True
delete_model = True
delete_bucket = False

#### **撤销模型和删除端点**

In [ ]:
if delete_endpoint:
    endpoint.delete(force=True)

删除模型

In [ ]:
if delete_model:
    model.delete()

从暂存存储桶中删除内容

请注意：此云存储桶中的所有内容将被删除。请谨慎操作。

In [ ]:
if delete_bucket and "BUCKET_NAME" in globals():
    print(f"Deleting all contents from the bucket {BUCKET_NAME}")

    shell_output = ! gsutil du -as $BUCKET_NAME
    print(
        f"Size of the bucket {BUCKET_NAME} before deleting = {shell_output[0].split()[0]} bytes"
    )

    # uncomment below line to delete contents of the bucket
    # ! gsutil rm -r $BUCKET_NAME

    shell_output = ! gsutil du -as $BUCKET_NAME
    if float(shell_output[0].split()[0]) > 0:
        print(
            "PLEASE UNCOMMENT LINE TO DELETE BUCKET. CONTENT FROM THE BUCKET NOT DELETED"
        )

### 清理笔记本环境

在实验完成后，您可以选择[停止](https://cloud.google.com/ai-platform/notebooks/docs/shut-down)或删除 AI 笔记本实例，以避免任何费用。如果想保存您的工作，可以选择停止实例。

```
# 停止笔记本实例
gcloud notebooks instances stop example-instance --location=us-central1-a


# 删除笔记本实例
gcloud notebooks instances delete example-instance --location=us-central1-a
```